In [ ]:
# Install required packages
!pip uninstall -y fsspec gcsfs
!pip install -q fsspec[http]==2024.6.1 gcsfs==2024.6.1
!pip install -q datasets transformers torch accelerate bitsandbytes

import os
import json
import torch
import pandas as pd
import numpy as np
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import Dataset
from typing import List, Dict, Tuple
from google.colab import drive

class MIRFlanT5TrainerQAFormat:
    def __init__(self,
                 base_model: str = "google/flan-t5-base",
                 output_dir: str = "mir_flan_t5_qa"):
        """Initialize the FLAN-T5 trainer for MIR questions"""
        print("Initializing MIR FLAN-T5 Trainer...")
        self.base_model = base_model
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)

        print(f"Loading base model: {base_model}")
        self.tokenizer = AutoTokenizer.from_pretrained(base_model)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(base_model)

        # Move model to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        self.model = self.model.to(self.device)

    def format_question(self, item: Dict) -> Dict:
        """Format a question for QA training"""
        try:
            # Get question text from input field (removing "Medical Question: " prefix)
            question_text = item['input'].replace("Medical Question: ", "").strip()
            correct_answer = item['output']

            input_text = f"""Responde esta pregunta de la manera más concisa posible.

Pregunta: {question_text}"""

            return {
                'input': input_text,
                'output': correct_answer
            }
        except Exception as e:
            print(f"Error formatting question: {str(e)}")
            print(f"Item structure: {json.dumps(item, indent=2)}")
            return None

    def prepare_training_data(self, questions: List[Dict]) -> Tuple[Dataset, Dataset]:
        """Prepare and split training data"""
        print("Preparing training data...")
        training_data = []

        for i, question in enumerate(questions):
            try:
                formatted = self.format_question(question)
                if formatted:
                    training_data.append(formatted)

                if (i + 1) % 100 == 0:
                    print(f"Processed {i + 1} questions...")

            except Exception as e:
                print(f"Error processing question {i}: {str(e)}")
                continue

        print(f"Created {len(training_data)} training examples")

        if len(training_data) == 0:
            raise ValueError("No valid training examples were created!")

        # Convert to DataFrame
        df = pd.DataFrame(training_data)

        # Tokenize inputs and outputs
        inputs = list(df["input"])
        outputs = list(df["output"])

        # Tokenize with padding and truncation
        tokenized_inputs = self.tokenizer(
            inputs, padding=True, truncation=True, max_length=512, return_tensors="pt"
        )
        tokenized_outputs = self.tokenizer(
            outputs, padding=True, truncation=True, max_length=128, return_tensors="pt"
        )

        # Create tokenized dataset
        tokenized_data = {
            "input_ids": tokenized_inputs["input_ids"],
            "attention_mask": tokenized_inputs["attention_mask"],
            "labels": tokenized_outputs["input_ids"],
        }

        # Split data
        total_examples = len(tokenized_data["input_ids"])
        train_size = int(0.9 * total_examples)

        train_data = {k: v[:train_size] for k, v in tokenized_data.items()}
        eval_data = {k: v[train_size:] for k, v in tokenized_data.items()}

        train_dataset = Dataset.from_dict(train_data)
        eval_dataset = Dataset.from_dict(eval_data)

        return train_dataset, eval_dataset

    def train(self, train_dataset: Dataset, eval_dataset: Dataset):
        """Train the model"""
        print("Starting training...")

        # Disable W&B integration
        os.environ["WANDB_DISABLED"] = "true"

        training_args = TrainingArguments(
            output_dir=self.output_dir,
            num_train_epochs=3,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            warmup_steps=100,
            weight_decay=0.01,
            logging_dir=f'{self.output_dir}/logs',
            logging_steps=10,
            evaluation_strategy="steps",
            eval_steps=100,
            save_strategy="steps",
            save_steps=100,
            load_best_model_at_end=True,
            save_total_limit=2,
            fp16=torch.cuda.is_available(),
            remove_unused_columns=False,
        )

        data_collator = DataCollatorForSeq2Seq(
            tokenizer=self.tokenizer,
            model=self.model,
            padding=True,
            max_length=512
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            data_collator=data_collator,
            tokenizer=self.tokenizer
        )

        print("Training model...")
        trainer.train()

        print("Saving model...")
        trainer.save_model()
        self.tokenizer.save_pretrained(self.output_dir)
        print(f"Model saved to {self.output_dir}")

def main():
    print("Starting MIR FLAN-T5 QA Format Training Pipeline...")

    try:
        # Mount Google Drive
        drive.mount('/content/drive')

        # Define paths
        base_dir = '/content/drive/MyDrive/TFM2'
        training_data_path = f"{base_dir}/meli-training-content/simple_qa/flan_t5_training.json"
        output_dir = f"{base_dir}/models/mir_flan_t5_qa"

        # Initialize trainer
        trainer = MIRFlanT5TrainerQAFormat(
            base_model="google/flan-t5-base",
            output_dir=output_dir
        )

        # Load training data
        print("\nLoading training data...")
        with open(training_data_path, 'r', encoding='utf-8') as f:
            questions = json.load(f)
        print(f"Loaded {len(questions)} questions")

        # Print sample question for debugging
        if questions:
            print("\nSample question format:")
            print(json.dumps(questions[0], indent=2))

        # Prepare and split training data
        train_dataset, eval_dataset = trainer.prepare_training_data(questions)

        print(f"\nSplit sizes:")
        print(f"Training examples: {len(train_dataset)}")
        print(f"Validation examples: {len(eval_dataset)}")

        # Train the model
        trainer.train(train_dataset, eval_dataset)

        print("\nTraining complete! Model saved to:", output_dir)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

Found existing installation: fsspec 2024.6.1
Uninstalling fsspec-2024.6.1:
  Successfully uninstalled fsspec-2024.6.1
Found existing installation: gcsfs 2024.6.1
Uninstalling gcsfs-2024.6.1:
  Successfully uninstalled gcsfs-2024.6.1
Starting MIR FLAN-T5 QA Format Training Pipeline...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Initializing MIR FLAN-T5 Trainer...
Loading base model: google/flan-t5-base
Using device: cuda

Loading training data...
Loaded 619 questions

Sample question format:
{
  "id": "MED2019-P105",
  "input": "Medical Question: 000 UIL. Se aporta imagen de la ecograf\u00eda transvaginal. Qu\u00e9 indicar\u00eda en ese momento:",
  "output": "Tratamiento con misoprostol por v\u00eda sist\u00e9mica. - 2 029102 ANICIDEM ESF",
  "context": {
    "year": "2019",
    "question_number": "105"
  }
}
Preparing training data...
Processed 100 questions...
Processed 200 questions...
Processed 300

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



Split sizes:
Training examples: 557
Validation examples: 62
Starting training...


<ipython-input-3-5c7200c75d82>:145: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training model...


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
100,0.000000,nan
200,0.000000,nan


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Saving model...
Model saved to /content/drive/MyDrive/TFM2/models/mir_flan_t5_qa

Training complete! Model saved to: /content/drive/MyDrive/TFM2/models/mir_flan_t5_qa
